<a href="https://colab.research.google.com/github/liz-lewis-manchester/CNM_2025_group_01/blob/testing-solver/tests/Update_Test_case_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import matplotlib.animation


class AdvectionSolver:
  #Finite difference solver for the 1D advection equation: d(theta)/dt + U*d(theta)/dx = 0


    def __init__(self, L, dx, dt, T, U):
        self.L = L
        self.dx = dx
        self.dt = dt
        self.T = T
        self.U = U

        # Spatial grid
        self.x_new = np.arange(0, L+dx, dx)
        self.num_points = len(self.x_new)
        self.num_time_steps = int(self.T / self.dt)

        if self.U <= 0:
            raise ValueError("This solver assumes flow velocity U > 0.")

    def _solve_implicit(self, theta_initial, boundary_theta):
        """
        Implicit Upwind Scheme (Standard BTBS).
        Unconditionally stable.
        Equation: theta_i^{n+1} = (theta_i^n + C * theta_{i-1}^{n+1}) / (1 + C)
        """
        print(f"Using **Implicit Scheme (Standard BTBS)**.")

        theta = np.copy(theta_initial)
        N = self.num_points

        theta_history = np.zeros((self.num_time_steps + 1, N))
        theta_history[0, :] = theta

        C = abs(self.U) * self.dt / self.dx # Parameter C for the implicit formula
        divisor = 1.0 + C

        for n in range(self.num_time_steps):
            theta_next = np.zeros_like(theta)

            # Boundary Condition
            theta_next[0] = boundary_theta

            # Forward Substitution
            for i in range(1, N):
                numerator = theta[i] + C * theta_next[i-1]
                theta_next[i] = numerator / divisor

            theta = theta_next
            theta_history[n+1, :] = theta

        return theta_history

    def compute_solution(self, theta_initial, boundary_theta):

        return self._solve_implicit(theta_initial, boundary_theta)


L = 20
T = 300
dx = 0.2
dt = 10
U = 0.1
boundary_theta = 250.0

x_new = np.arange(0, L + dx, dx)
theta_initial = np.zeros_like(x_new)
theta_initial[0] = boundary_theta
#solve
solver_A = AdvectionSolver(L, dx, dt, T, U)
res_A = solver_A.compute_solution(theta_initial, boundary_theta)

fig, ax = plt.subplots(figsize=(4,3))

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.ioff() # interactive off

def animate(t):
  plt.cla()
  y = res_A[t]
  plt.plot(x_new, y)
  plt.xlim(0,L)
  plt.ylim(0, 300)
  ax.set_title("Concentration of pollutant at t = " + str(dt*t) + "s", fontsize=10, verticalalignment='top')
  plt.xlabel("x (m)", fontsize=8)
  plt.ylabel("Concentration (µg/m³)", fontsize=8)

anim = matplotlib.animation.FuncAnimation(fig, animate, frames=len(res_A))
anim

Using **Implicit Scheme (Standard BTBS)**.
